In [1]:
import numpy as np # arrays & loading data
import tensorflow as tf # for building neural networks
from tensorflow.keras.models import Sequential  # model type that we will use
from tensorflow.keras.layers import Dense # we will use Dense layers
from sklearn.preprocessing import StandardScaler # z-score normalization 

# suppress warnings
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

In [2]:
# unpickle the data from the batch files in the CIFAR-10 dataset
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        data_batch_data = pickle.load(fo, encoding='bytes')
    return data_batch_data

# here we will add all the batches of data
all_data=np.empty((1,3073))


for i in range(1,6):
    # save the data from each batch as a dictionary
    data_batch_data = unpickle(f"cifar-10-batches-py\\data_batch_{i}")

    # split the data using the keys
    labels = np.array(data_batch_data[b'labels'])
    data = np.array(data_batch_data[b'data'])

    # form a column vector where only 0 and 1 are kept ( we need binary classification)
    labels_zero_or_one = np.where((labels<2),labels,-1)
    labels_zero_or_one = labels_zero_or_one.reshape(-1,1) 
    
    # concatanate the previous labels column to the data
    data_concatanated = np.concatenate((data,labels_zero_or_one),axis=1)
    
    # keep only the rows that have 0 or 1 as labels (those with -1 as labels are not important now)
    data_final= data_concatanated[data_concatanated[:,-1]!=-1]

    all_data= np.concatenate((all_data,data_final),axis=0)
    
all_data=all_data[1:] # remove the first row (it contains unimportant elements)

print(all_data)
print(all_data.shape)

[[170. 168. 177. ...  78.  80.   1.]
 [159. 150. 153. ...  17.  19.   1.]
 [202. 202. 204. ... 243. 243.   0.]
 ...
 [156. 155. 156. ... 162. 162.   0.]
 [189. 186. 185. ... 171. 171.   1.]
 [229. 236. 234. ... 162. 161.   1.]]
(10000, 3073)


In [3]:
X = all_data[:,:-1] # forming the input and output 
y = all_data[:,-1]

y = np.expand_dims(y, axis=1) # make y 2D - the commands later will require it

# split the data into TRAINING, CROSS-VALIDATION 
from sklearn.model_selection import train_test_split

# TRAINING SET - 80%, CV SET - 20%
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.20, random_state=1)


print(f"training input shape:{X_train.shape}")
print(f"training output shape:{y_train.shape}")
print(f"cv input shape:{X_cv.shape}")
print(f"cv output shape:{y_cv.shape}")




training input shape:(8000, 3072)
training output shape:(8000, 1)
cv input shape:(2000, 3072)
cv output shape:(2000, 1)


In [4]:
# each image has 3072 units (this is explained in more detail in the documentation in the link provided)

# sequential model structure

# we may CHANGE TO CONVOLUTIONAL in the future, as it is more efficient for images
model= Sequential(
    [
        tf.keras.Input(shape=(3072,)), # input size
        Dense(200,activation="sigmoid", name="layer1"),
        Dense(120,activation="sigmoid", name="layer2"),
        Dense(60,activation="sigmoid", name="layer3"),
        Dense(15,activation="sigmoid", name="layer4"),
        Dense(1,activation="sigmoid", name="layer5"),
    ], name="binary_model"
)

In [5]:
# see details about the activation of every layer and the form of the w and b parameters
model.summary()

Model: "binary_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                  │ (None, 200)            │       614,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer2 (Dense)                  │ (None, 120)            │        24,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer3 (Dense)                  │ (None, 60)             │         7,260 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer4 (Dense)                  │ (None, 15)             │           915 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer5 (Dense)                  │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 646,911 (2.47 MB)

 Trainable params: 646,911 (2.47 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# applying z-score to all the training data & cv data - adjust the data based on its distribution for Adam to converge faster
standard_scaler = StandardScaler()
X_train_scaled = standard_scaler.fit_transform(X_train)
X_cv_scaled = standard_scaler.transform(X_cv) 

# define loss and optimizer of the Adam's algorithm
model.compile(
    # this is similar to gradient descent, but it is a much improved version
    loss=tf.keras.losses.BinaryCrossentropy(), # BC - binary class 
    optimizer=tf.keras.optimizers.Adam(0.01), # preimplemented optimizer
)

In [10]:
# train the model "epochs" times
model.fit(
    X_train_scaled, y_train,
    epochs = 10
    
    
)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2879
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.3007
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2991
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.2968
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2884
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2752
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2877
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2932
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2960
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.2906


In [11]:
# fraction of misclassified outputs in the TRAINING SET
y_predicted = model.predict(X_train_scaled)
# classic way of calculating the error in a classification case
y_predicted = np.where(y_predicted>=0.5,1,0)
error_training = np.mean(y_predicted != y_train) # arithmetic mean, where the numerator is the count of wrong predictions
print(f"Training Set Classification Error: {error_training}")

# do the same for CV set
y_predicted = model.predict(X_cv_scaled)
y_predicted = np.where(y_predicted>=0.5,1,0)
error_cv = np.mean(y_predicted != y_cv) 
print(f"CV Set Classification Error: {error_cv}")

250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Training Set Classification Error: 0.1205
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
CV Set Classification Error: 0.1705


In [12]:
# save the test batch data 
test_batch_data = unpickle("cifar-10-batches-py\\test_batch")

# split the data using the keys
labels = np.array(test_batch_data[b'labels'])
data = np.array(test_batch_data[b'data'])

# form a column vector where only 0 and 1 are kept ( we need binary classification)
labels_zero_or_one = np.where((labels<2),labels,-1)
labels_zero_or_one = labels_zero_or_one.reshape(-1,1) 

# concatanate the labels column form above to the data
data_concatanated = np.concatenate((data,labels_zero_or_one),axis=1)

# keep only the rows that have 0 or 1 as labels (those with -1 as labels are not important now)
data_final= data_concatanated[data_concatanated[:,-1]!=-1]



In [13]:
X = data_final[:,:-1] # forming the input and output 
y = data_final[:,-1]

y_test = np.expand_dims(y, axis=1) # make y 2D - the commands later will require it

In [14]:
X_test_scaled = standard_scaler.transform(X) 

In [16]:
# fraction of misclassified outputs in the TEST SET
y_predicted = model.predict(X_test_scaled)
y_predicted = np.where(y_predicted>=0.5,1,0)
error_test = np.mean(y_predicted != y_test) 
print(f"Test Set Classification Error: {error_test}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Test Set Classification Error: 0.1615
